### Synopsis
This code calculates the absolute cross-section coefficient using the 1) metadata in the result files and 2) linear fitting coefficients in IC-to-photons conversion.

Theory: Sheyfer et al., Phys. Rev. Lett. 125, 125504 (2020), Eq. S1

### $I(Q) = (F_sT_st_{exp}f\Delta\Omega)^{-1}I_s(Q) - (F_{bg}T_{bg}t_{exp}f\Delta\Omega)^{-1}I_{bg}(Q)$

Here:

$F_s, F_{bg}$  : Incoming x-ray flux for the sample and the background (buffer sample).  
$T_s, T_{bg}$  : Transmission coefficient for the sample and the buffer.  
$t_{exp}$  : Exposure time. Note that there is a time mismatch between I(Q) (photon per frame) and measured flux (photon per second).  
$f$  : Sample thickness.  
$\Delta\Omega$  : Solid angle spanned by a single detector pixel.   


$(F_sT_st_{exp}f\Delta\Omega)^{-1}$  : Absolute scattering cross-section coefficient for the sample.  
$(F_{bg}T_{bg}t_{exp}f\Delta\Omega)^{-1}$  : Absolute scattering cross-section coefficient for the buffer.

In [1]:

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import os
from scipy.optimize import curve_fit

import h5py


In [2]:

mount_point = '/gdata/s8id-dmdtn/'

fn_dir = mount_point + '2022-1/babnigg202203/cluster_results_QZ/'
fn = 'E0110_S3_4_Ramp_340C10p_att00_Rq0_00001_0001-100000.hdf'

with h5py.File(fn_dir+fn, 'r') as HDF_Result:
    det_dist = np.squeeze(HDF_Result.get('/measurement/instrument/detector/distance')[()])
    pix_dim_x = np.squeeze(HDF_Result.get('/measurement/instrument/detector/x_pixel_size')[()])
    pix_dim_y = np.squeeze(HDF_Result.get('/measurement/instrument/detector/x_pixel_size')[()])
    Up_IC = np.squeeze(HDF_Result.get('/measurement/instrument/source_end/I0Monitor')[()])
    Dn_IC = np.squeeze(HDF_Result.get('/measurement/instrument/source_end/TransmissionMonitor')[()])
    sample_thickness = np.squeeze(HDF_Result.get('/measurement/sample/thickness')[()])

    num_frames = np.squeeze(HDF_Result.get('/xpcs/data_end')[()])
    t0 = np.squeeze(HDF_Result.get('/measurement/instrument/detector/exposure_period')[()])

t_exp = t0*num_frames
F_ec = 6.25e4*(Up_IC/t_exp)-1.28e7
T_ec = Dn_IC/Up_IC/0.868
f = sample_thickness
Delta_Omega = (pix_dim_x/det_dist)*(pix_dim_y/det_dist)

coef_sam = num_frames/(t_exp*F_ec*T_ec*f*Delta_Omega)


In [3]:

mount_point = '/gdata/s8id-dmdtn/'

fn_dir = mount_point + '/2022-1/babnigg202203/cluster_results_QZ/'
fn = 'D0138_Buffer_034C_att00_Rq0_00001_0001-100000.hdf'

with h5py.File(fn_dir+fn, 'r') as HDF_Result:
    det_dist = np.squeeze(HDF_Result.get('/measurement/instrument/detector/distance')[()])
    pix_dim_x = np.squeeze(HDF_Result.get('/measurement/instrument/detector/x_pixel_size')[()])
    pix_dim_y = np.squeeze(HDF_Result.get('/measurement/instrument/detector/x_pixel_size')[()])
    Up_IC = np.squeeze(HDF_Result.get('/measurement/instrument/source_end/I0Monitor')[()])
    Dn_IC = np.squeeze(HDF_Result.get('/measurement/instrument/source_end/TransmissionMonitor')[()])
    sample_thickness = np.squeeze(HDF_Result.get('/measurement/sample/thickness')[()])

    num_frames = np.squeeze(HDF_Result.get('/xpcs/data_end')[()])
    t0 = np.squeeze(HDF_Result.get('/measurement/instrument/detector/exposure_period')[()])

t_exp = t0*num_frames
F_ec = 6.25e4*(Up_IC/t_exp)-1.28e7
T_ec = Dn_IC/Up_IC/0.868
f = sample_thickness
Delta_Omega = (pix_dim_x/det_dist)*(pix_dim_y/det_dist)

coef_buf = num_frames/(t_exp*F_ec*T_ec*f*Delta_Omega)


In [4]:

print(f'Absolute scattering cross-section coefficient for the sample is {coef_sam:.2e}')
print(f'Absolute scattering cross-section coefficient for the sample is {coef_buf:.2e}')


Absolute scattering cross-section coefficient for the sample is 6.93e+04
Absolute scattering cross-section coefficient for the sample is 7.62e+04
